<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/06_PyTorch_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch入門（2）


## 線形回帰モデル
* 線形回帰モデルのmini-batch gradient descentをPyTorchで書いてみる。
* PyTorchのDatasetとDataLoaderの使い方も併せて学ぶ。
* ついでにwandbも使ってみます。

## 準備

* ランタイムのタイプは、今回はCPUで構わないです。

* reproducibilityについては下記リンク先を参照
  * https://pytorch.org/docs/stable/notes/randomness.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import torch.nn as nn
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader

%config InlineBackend.figure_format='retina'

np.random.seed(0)
torch.manual_seed(0)

## synthetic data
* $y = w_1 x_1 + w_2 x_2 + b + \epsilon$という式にしたがってデータを生成する。
  * $\epsilon$は正規分布に従うとする。

In [ ]:
# データ数
data_size = 1000

# ランダムな二次元ベクトルの集合を訓練データとして設定
X = 10 * torch.rand(data_size, 2) - 5.0

# 係数と切片の正解（これに近い値が求まればよい）
w_true = torch.tensor([[2.0], [-3.0]])
b_true = torch.tensor([10.0])

# 正規乱数を加えた値がターゲット
y = X @ w_true + b_true + torch.normal(0.0, 2.0, (data_size, 1))

## Datasetクラス
* `torch.utils.data.Dataset`を継承して自分用のデータセットのクラスを定義する。
* 以下の２つの関数を必ず書く。
  * データセットの長さを返す関数`__len__`
  * 与えられたインデックスに対応するアイテムを返す関数`__getitem__`

In [ ]:
class MyDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]

### データセットの分割
* ここでは train : valid : test = 8 : 1 : 1 に分割することにする。
  * この分割の比率に、深い意味はない。

In [ ]:
dataset = MyDataset(X, y)
train, val, test = random_split(dataset, [0.8, 0.1, 0.1])

In [ ]:
print(f"training size: {len(train)}, validation size: {len(val)}, test size:{len(test)}")

* 分割した後は、`torch.utils.data.Dataset`ではなく、`torch.utils.data.dataset.Subset`になる。

In [ ]:
type(train)

In [ ]:
train[:5]

### 訓練データを可視化

In [ ]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=30, azim=45)

ax.scatter(train[:][0][:,0], train[:][0][:,1], train[:][1], c='b', marker='o')

ax.set_xlabel('X1 axis')
ax.set_ylabel('X2 axis')
ax.set_zlabel('Y axis')
ax.set_title('3D Scatter Plot of Training Set');

## DataLoaderクラス
* 訓練データをシャッフルしてミニバッチをひとつずつ取り出す処理を、PyTorchのDataLoaderを使って実装する。
  * https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader

In [ ]:
batch_size = 10

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

* データローダの長さは、ミニバッチの個数。インスタンスの個数ではない。

In [ ]:
len(train_loader)

* 訓練データの最初のミニバッチだけ見てみる。

In [ ]:
print(next(iter(train_loader)))

## モデルの定義と初期化
* 値を推定したいのは、線形モデル$y = w_1 x_1 + w_2 x_2 + b$の係数$w_1,w_2$と切片$b$。
* そこで、係数と切片を微分可能なテンソルとして用意する。

In [ ]:
w = torch.randn((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [ ]:
print(w)
print(b)

## 損失関数
* 平均二乗誤差を使う。
* PyTorchで用意されている損失関数については、下のリンク先を参照。
 * https://pytorch.org/docs/stable/nn.html#loss-functions

In [ ]:
criterion = torch.nn.MSELoss()

## 最適化アルゴリズム
* 今回は、SGDを使う。
 * 説明のため、あえて`momentum`は使わない。
* PyTorchで用意されているoptimizerについては、下のリンク先を参照。
 * https://pytorch.org/docs/stable/optim.html#algorithms

In [ ]:
optimizer = torch.optim.SGD([w, b], lr=0.001)

## 評価用のヘルパ関数

* 評価する際には、計算グラフを作る必要はない。
* `with torch.no_grad()`で計算グラフを作らないようにする。

In [ ]:
def evaluate(loader, w, b):
  total_size = 0
  total_loss = 0.0
  for input, target in loader:
    with torch.no_grad():
      output = input @ w + b
      loss = criterion(output, target)
      total_size += len(target)
      total_loss += loss.item() * len(target)
  return total_loss, total_size

## 学習
* ループの内側には、以下の４つを書く
1. 損失関数の値を計算することによって、計算グラフを作る
2. backpropagationの実行
3. パラメータの更新
4. gradientをゼロにする



* コメントアウトしたprint関数を実行させると・・・
 * 本当に勾配を使ってwを更新していることが分かる。

In [ ]:
epoch = 0
step = 0

In [ ]:
for _ in range(10):
  epoch += 1

  train_size = 0
  train_loss = 0.0
  for input, target in train_loader:
    step += 1

    output = input @ w + b
    loss = criterion(output, target)
    train_size += len(target)
    train_loss += loss.item() * len(target) # ロギング用の集計

    #print(f"\t step {step}: w before update {list(w.detach().numpy().flatten())}")
    loss.backward()
    #print(f"\t\t w.grad {list(w.grad.detach().numpy().flatten())}")
    optimizer.step()
    #print(f"\t step {step}: w after update {list(w.detach().numpy().flatten())}")
    #print("-"*60)
    optimizer.zero_grad()

  # validation lossの計算
  val_loss, val_size = evaluate(val_loader, w, b)

  # print関数でロギング
  print(f'epoch {epoch}: ',
        f'train loss {train_loss/train_size:.4f} ',
        f'validation loss {val_loss/val_size:.4f} ',
        f'w={list(w.detach().numpy().flatten())}',
        f'b={b.item():.4f}')
  print("="*90)


In [ ]:
test_loss, test_size = evaluate(test_loader, w, b)
print(f"test loss {test_loss / test_size:8.4f}")

* 推定された回帰平面を可視化してみる。

In [ ]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=30, azim=45)

ax.scatter(train[:][0][:,0], train[:][0][:,1], train[:][1], c='b', marker='o')

x1_min, x2_min = train[:][0].min(0)[0]
x1_max, x2_max = train[:][0].max(0)[0]

x1 = np.linspace(x1_min, x1_max, 100)
x2 = np.linspace(x2_min, x2_max, 100)
x1, x2 = np.meshgrid(x1, x2)

y = w[0].item() * x1 + w[1].item() * x2 + b.item()
ax.plot_surface(x1, x2, y, alpha=0.5, rstride=100, cstride=100, color='r', edgecolor='none')

ax.set_xlabel('X1 axis')
ax.set_ylabel('X2 axis')
ax.set_zlabel('Y axis')
ax.set_title('3D Plot of Regression Plane');

## wandb (Weights&Biases) を使う

* https://www.wandb.jp/

* 参考資料
  * 本家のquickstart https://docs.wandb.ai/quickstart
  * 解説記事の例 https://zenn.dev/zenizeni/books/a64578f98450c2/viewer/e18fdf

### インストール

In [ ]:
!pip install wandb

### ログイン

* APIキーを取得する。
  * https://wandb.ai/authorize

In [ ]:
import wandb
wandb.login()

### wandbの初期化

In [ ]:
batch_size = 10
learning_rate = 0.001
epochs = 100

run = wandb.init(
    # プロジェクト名
    project="my-linear-regression-project",
    # トラックするハイパーパラメータや実行のメタデータ
    config={
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "epochs": epochs,
    },
)

### 学習の準備
* wandbには関係なく、上で書いたことをもう一度まとめて書いているだけ。

In [ ]:
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

w = torch.randn((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD([w, b], lr=learning_rate)

### 学習
* wandbでログをとっている。

In [ ]:
for epoch in range(epochs):

  train_size = 0
  train_loss = 0.0
  for input, target in train_loader:
    output = input @ w + b
    loss = criterion(output, target)
    train_size += len(target)
    train_loss += loss.item() * len(target)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  wandb.log({"loss/training": train_loss / train_size})
  val_loss, val_size = evaluate(val_loader, w, b)
  wandb.log({"loss/validation": val_loss / val_size})

### 結果を見に行く
* https://wandb.ai/home

## nn.Sequentialクラス
* requires_grad=Trueでテンソルを作ればモデルを用意することはできる。
* しかし、同じことは、torch.nnを使えばもっとすっきり実現できる。
* まず、nn.Sequentialを使う方法を示す。

### nn.Sequentialのインスタンスとしてモデルを作る
* 下記のようにモデルを作った時点でレイヤのパラメータは初期化されている。
* この初期化には上でセットした乱数のシードが使われている。

In [ ]:
model = nn.Sequential(
    nn.Linear(2, 1),
)

In [ ]:
type(model)

In [ ]:
print(model)

* パラメータがどのように初期化されているかを確認してみる

In [ ]:
for name, p in model.named_parameters():
  print(name, p.data)

### 学習のループ

In [ ]:
batch_size = 10
learning_rate = 0.001
epochs = 100

run = wandb.init(
    # プロジェクト名
    project="my-linear-regression-project",
    # トラックするハイパーパラメータや実行のメタデータ
    config={
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "epochs": epochs,
    },
)

In [ ]:
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def evaluate(loader, model):
  total_size = 0
  total_loss = 0.0
  for input, target in loader:
    with torch.no_grad():
      output = model(input)
      loss = criterion(output, target)
      total_size += len(target)
      total_loss += loss.item() * len(target)
  return total_loss, total_size

In [ ]:
for epoch in range(epochs):

  train_size = 0
  train_loss = 0.0
  for input, target in train_loader:
    output = model(input)
    loss = criterion(output, target)
    train_size += len(target)
    train_loss += loss.item() * len(target)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  wandb.log({"loss/training": train_loss / train_size})
  val_loss, val_size = evaluate(val_loader, model)
  wandb.log({"loss/validation": val_loss / val_size})

## nn.Moduleクラス
* nn.Moduleを継承するクラスを定義する。
* そしてそのクラスのインスタンスとしてモデルを作る。

In [ ]:
class MyLinearModel(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.fc = nn.Linear(input_size, output_size)

  def forward(self, input):
    return self.fc(input)

In [ ]:
model = MyLinearModel(2, 1)

In [ ]:
for name, p in model.named_parameters():
  print(name, p.data)

In [ ]:
batch_size = 10
learning_rate = 0.001
epochs = 100

run = wandb.init(
    # プロジェクト名
    project="my-linear-regression-project",
    # トラックするハイパーパラメータや実行のメタデータ
    config={
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "epochs": epochs,
    },
)

In [ ]:
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):

  train_size = 0
  train_loss = 0.0
  for input, target in train_loader:
    output = model(input)
    loss = criterion(output, target)
    train_size += len(target)
    train_loss += loss.item() * len(target)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  wandb.log({"loss/training": train_loss / train_size})
  val_loss, val_size = evaluate(val_loader, model)
  wandb.log({"loss/validation": val_loss / val_size})

# 予告
* 次の回では、下のPyTorchのチュートリアルを使います。
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html